In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#GPU Libraries
import cudf as pd
import cupy as np
import cuml

In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
train.head()

In [4]:
test.head()

In [5]:
train.info()

In [6]:
test.info()


# EDA

In [7]:
Y = train['SalePrice']
train.drop('SalePrice',axis=1,inplace=True)

In [8]:
# handling missing values by either dropping the column or by taking mean or mode. 
train.isnull().sum()

In [9]:
test.isnull().sum()

In [10]:
train['LotFrontage']=train['LotFrontage'].fillna(train['LotFrontage'].mean())
train['LotFrontage'].value_counts()
train['BsmtCond']=train['BsmtCond'].fillna(train['BsmtCond'].mode()[0])
train['BsmtQual']=train['BsmtQual'].fillna(train['BsmtQual'].mode()[0])
train['FireplaceQu']=train['FireplaceQu'].fillna(train['FireplaceQu'].mode()[0])
train['GarageType']=train['GarageType'].fillna(train['GarageType'].mode()[0])
train['GarageFinish']=train['GarageFinish'].fillna(train['GarageFinish'].mode()[0])
train['GarageQual']=train['GarageQual'].fillna(train['GarageQual'].mode()[0])
train['GarageCond']=train['GarageCond'].fillna(train['GarageCond'].mode()[0])
train['MasVnrType']=train['MasVnrType'].fillna(train['MasVnrType'].mode()[0])
train['MasVnrArea']=train['MasVnrArea'].fillna(train['MasVnrArea'].mode()[0])
train['BsmtExposure']=train['BsmtExposure'].fillna(train['BsmtExposure'].mode()[0])
train['BsmtFinType2']=train['BsmtFinType2'].fillna(train['BsmtFinType2'].mode()[0])
train['Utilities']=train['Utilities'].fillna(train['Utilities'].mode()[0])
train['Exterior1st']=train['Exterior1st'].fillna(train['Exterior1st'].mode()[0])
train['Exterior2nd']=train['Exterior2nd'].fillna(train['Exterior2nd'].mode()[0])
train['BsmtFinType1']=train['BsmtFinType1'].fillna(train['BsmtFinType1'].mode()[0])
train['SaleType']=train['SaleType'].fillna(train['SaleType'].mode()[0])
train['BsmtFullBath']=train['BsmtFullBath'].fillna(train['BsmtFullBath'].mode()[0])
train['BsmtHalfBath']=train['BsmtHalfBath'].fillna(train['BsmtHalfBath'].mode()[0])
train['KitchenQual']=train['KitchenQual'].fillna(train['KitchenQual'].mode()[0])
train['Functional']=train['Functional'].fillna(train['Functional'].mode()[0])
train['GarageCars']=train['GarageCars'].fillna(train['GarageCars'].mean())
train['GarageArea']=train['GarageArea'].fillna(train['GarageArea'].mean())
train['BsmtFinSF1']=train['BsmtFinSF1'].fillna(train['BsmtFinSF1'].mean())
train['BsmtFinSF2']=train['BsmtFinSF2'].fillna(train['BsmtFinSF2'].mean())
train['BsmtUnfSF']=train['BsmtUnfSF'].fillna(train['BsmtUnfSF'].mean())
train['TotalBsmtSF']=train['TotalBsmtSF'].fillna(train['TotalBsmtSF'].mean())

In [11]:
test['LotFrontage']=test['LotFrontage'].fillna(test['LotFrontage'].mean())
test['LotFrontage'].value_counts()
test['BsmtCond']=test['BsmtCond'].fillna(test['BsmtCond'].mode()[0])
test['BsmtQual']=test['BsmtQual'].fillna(test['BsmtQual'].mode()[0])
test['FireplaceQu']=test['FireplaceQu'].fillna(test['FireplaceQu'].mode()[0])
test['GarageType']=test['GarageType'].fillna(test['GarageType'].mode()[0])
test['GarageFinish']=test['GarageFinish'].fillna(test['GarageFinish'].mode()[0])
test['GarageQual']=test['GarageQual'].fillna(test['GarageQual'].mode()[0])
test['GarageCond']=test['GarageCond'].fillna(test['GarageCond'].mode()[0])
test['MasVnrType']=test['MasVnrType'].fillna(test['MasVnrType'].mode()[0])
test['MasVnrArea']=test['MasVnrArea'].fillna(test['MasVnrArea'].mode()[0])
test['BsmtExposure']=test['BsmtExposure'].fillna(test['BsmtExposure'].mode()[0])
test['BsmtFinType2']=test['BsmtFinType2'].fillna(test['BsmtFinType2'].mode()[0])
test['MSZoning']=test['MSZoning'].fillna(test['MSZoning'].mode()[0])
test['Utilities']=test['Utilities'].fillna(test['Utilities'].mode()[0])
test['Exterior1st']=test['Exterior1st'].fillna(test['Exterior1st'].mode()[0])
test['Exterior2nd']=test['Exterior2nd'].fillna(test['Exterior2nd'].mode()[0])
test['BsmtFinType1']=test['BsmtFinType1'].fillna(test['BsmtFinType1'].mode()[0])
test['SaleType']=test['SaleType'].fillna(test['SaleType'].mode()[0])
test['BsmtFullBath']=test['BsmtFullBath'].fillna(test['BsmtFullBath'].mode()[0])
test['BsmtHalfBath']=test['BsmtHalfBath'].fillna(test['BsmtHalfBath'].mode()[0])
test['KitchenQual']=test['KitchenQual'].fillna(test['KitchenQual'].mode()[0])
test['Functional']=test['Functional'].fillna(test['Functional'].mode()[0])
test['GarageCars']=test['GarageCars'].fillna(test['GarageCars'].mean())
test['GarageArea']=test['GarageArea'].fillna(test['GarageArea'].mean())
test['BsmtFinSF1']=test['BsmtFinSF1'].fillna(test['BsmtFinSF1'].mean())
test['BsmtFinSF2']=test['BsmtFinSF2'].fillna(test['BsmtFinSF2'].mean())
test['BsmtUnfSF']=test['BsmtUnfSF'].fillna(test['BsmtUnfSF'].mean())
test['TotalBsmtSF']=test['TotalBsmtSF'].fillna(test['TotalBsmtSF'].mean())

In [12]:
train.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)
test.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)

In [13]:
train.isnull().sum()

In [14]:
train=train.fillna(method ='pad')

In [15]:
df=pd.concat([train,test],axis=0)

In [16]:
df=pd.get_dummies(df)

In [17]:
df.head()

In [18]:
#Splitting the dataset into train and test
num_train,num_test = train.shape[0],test.shape[0]
train = df.iloc[:num_train,:]
test = df.iloc[num_test+1:,:]

In [19]:
X=train
X.shape

In [20]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.29,random_state=42)

In [21]:
from cuml import LinearRegression
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
split=lr.fit(X,Y)

In [22]:
print("R-Squared Value for Training Set: {:.3f}".format(lr.score(X_train.astype('float32'), y_train.astype('float32'))))

In [24]:
y_pred_split=lr.predict(test)
y_pred_split

In [25]:
#Evaluation Metrics for all 5 algorithms

# svd
print("Algorithm: svd")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_test)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_test.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),prediction.astype('int64')))

In [26]:
# eig
print("Algorithm: eig")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_test)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_test.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),prediction.astype('int64')))

In [27]:
# svd-qr
print("Algorithm: svd-qr")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-qr')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_test)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_test.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),prediction.astype('int64')))

In [28]:
# svd-jacobi
print("Algorithm: svd-jacobi")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_test)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_test.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),prediction.astype('int64')))

In [ ]:
# qr
print("Algorithm: qr")
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'qr')
reg = lr.fit(X_train,y_train)
prediction = lr.predict(X_test)
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),prediction.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(y_test.astype('float32'),prediction.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),prediction.astype('int64')))

In [30]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

In [31]:
sample_sub['SalePrice']

# Ridge Regression


In [32]:
from cuml import Ridge
alpha = np.array([1e-5])
r = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "eig")
model1=r.fit(X,Y)
y_pred_ridge = model1.predict(test)
y_pred_ridge

In [33]:
alpha = np.array([1e-5])
r2 = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "svd")
model2 = r2.fit(X,Y)
y_pred_ridge2 = model2.predict(test)
y_pred_ridge2

In [ ]:
alpha = np.array([1e-5])
r3 = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = "cvd")
model3 = r3.fit(X,Y)
y_pred_ridge3 = model3.predict(test)
y_pred_ridge3

# Lasso Regression

In [35]:
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.1)
result = ls.fit(X, Y)
y_pred_lasso=result.predict(test)
y_pred_lasso